# Named entity recognition with an LLM

Geoff Ford  
[https://geoffford.nz](https://geoffford.nz/)  

Consult the [README](README.md) for detailed information and the [CHANGELOG](CHANGELOG.md) for changes.

Run this cell to install required Python libraries.

In [ ]:
!pip install -r requirements.txt

Run the following cell to import relevant Python libraries used in this notebook and set the logging level.

In [1]:
import logging
import requests
import json
import getpass

logging.basicConfig(level=logging.INFO)

The [README](README.md) file discusses how to generate an OpenRouter API key. Configure the key by running this cell ...

In [2]:
OPENROUTER_API_KEY = getpass.getpass()

The following cell contains a function to query Open Router and generate LLM text. Just run it to make the function available. Change it if you know what you are doing.

Note: you can use API endpoints compatible with the OpenAI completion endpoint, but you will need to specify the relevant `api_url` and specify an `api_url` to `query_llm` calls. For example, if you have software to run LLMs locally, like [Ollama](https://ollama.com/), you can specifying an `api_url` (e.g. `http://127.0.0.1:11434/v1/chat/completions`).

In [3]:
def query_llm(prompt:str, # prompt to send to LLM
            model: str, # model name e.g. google/gemma-2-9b-it:free
            system_prompt: str = None, # system prompt to send to LLM
            max_tokens: int = 2048, # maximum number of tokens to generate (includes prompt tokens)
            response_format: str = None, # response format: json or None
            temperature: float = None, # temperature for sampling
            api_url: str = None # OpenAI completion endpoint compatible API to query, defaults to OpenRouter's API 
            ) -> str: # generated text from LLM call
    """ Query LLM with prompt """

    if api_url is None or api_url.strip() == '':
        api_url = "https://openrouter.ai/api/v1/chat/completions"
    
    if OPENROUTER_API_KEY is None:
        logging.error("OPENROUTER_API_KEY not set. Not querying llm.")
        return None
    api_key = OPENROUTER_API_KEY
    
    if prompt.strip() == '':
        logging.error('No prompt provided. Not querying llm.')
        return None
    
    messages = []
    if system_prompt is not None and system_prompt.strip() != '':
        messages.append({"role": "system", "content": system_prompt})
    messages.append({"role": "user", "content": prompt})

    request_data = {
                "model": model, 
                "messages": messages,
                'max_tokens': max_tokens
                    }

    if temperature is not None:
        request_data['temperature'] = temperature
    
    if response_format == "json":
        request_data['response_format'] = {"type": "json_object"}
        
    text = None

    try:
        response = requests.post(
            url=api_url,
            headers={
                "Authorization": f"Bearer {api_key}",
            },
            data=json.dumps(request_data)
            )
        response.raise_for_status() 
        text = response.json()['choices'][0]['message']['content']
    except requests.exceptions.RequestException as e:
        logging.error(f"Error querying LLM: {e}")
        print(response.json())
        raise
    except KeyError as e:
        logging.error(f"Error querying LLM: {e}")
        print(response.json())
        raise
    except Exception as e:
        logging.error(f"Error querying LLM: {e}")
        print(response.json())
        raise

    return text


## Set the model (and a note about OpenRouter free models)

When using OpenRouter's free models it is possible that specific models will be unavailable at times. If you get errors querying OpenRouter you can look up the [message or error codes in their documentation](https://openrouter.ai/docs). The `query_llm` function will raise errors when the API responds with an error code or if the JSON data returned by the API does not include generated content. If you get an error when using a free model, it is likely that this is temporary. Changing to [another free model](https://openrouter.ai/models?max_price=0) will typically resolve the issue. Look for models with '(free)' in their name. The next cell is where you can set the model to use for generation.

In [6]:
model = 'meta-llama/llama-3-8b-instruct:free'

## Task: Named entity extraction with an LLM

### Note about the temperature setting

The only parameter implemented in the `query_llm` function is `temperature` (([video](https://www.youtube.com/watch?v=ezgqHnWvua8)). Feel free to implement [other parameters available in OpenRouter's API](https://openrouter.ai/docs/parameters) if you are confident doing this, but this is not expected for class activities or supported.  

Lower temperature values give similiar or identical responses. Higher values produce more varied responses.
The default value of temperature is 1.0. It can vary between 0.0 and 2.0.

### Come up with a system_prompt and provide the input text

Your task is to come up with an appropriate system prompt to extract named entities from text supplied in the prompt. 

The system prompt currently only specifies to generate JSON data and has no instruction about named entity recognition. The system prompt can be improved by adding further instructions, including specifying the types of named entities to return. The suggested JSON format can also be altered, but the code in the next cells relies on returning a JSON array with `entities` as the key.

The lab task asks you to run named entity extraction on specific texts. Paste the text into the prompt.

For an application like named entity recognition, the temperature should be set to a low value to ensure less variation in responses.

You can change the model above, but use one of the OpenRouter free models. Getting good quality output is more challenging with a smaller model. 

In [23]:
system_prompt = '''

Only reply with JSON data.

JSON format:
{
    "entities": [
    ]
}

INPUT:
'''

prompt = '''
This text should be replaced with the texts you want to extract entities from. For now it is just a placeholder.
Here is a sentence to extract entities from: Hi there, I am Joe and I live in Christchurch. 
'''

max_tokens = 10000

temperature = 0.1

response_format = 'json'

Running the next cell queries the API and outputs a response. If the response is valid JSON you should get a list of entities. Otherwise, you will see the full response.

In [ ]:
response = query_llm(prompt, model, system_prompt, max_tokens, response_format = response_format, temperature = temperature)

try:
    json_data = json.loads(response)
    for i, entity in enumerate(json_data['entities']):
        print(entity)
except json.JSONDecodeError as e:
    print('Error decoding JSON. Try regenerating. A lower temperature value may help.')
    print('Here is the response that was returned:')
    print(response)
except KeyError as e:
    print('Error decoding JSON. An array with key "entities" is expected, but was not part of the JSON output.')
    print('Here is the response that was returned:')
    print(response)
    
